In [1]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
import math
from pytorch_lightning.loggers import WandbLogger
from tqdm import tqdm
import numpy as np
import torch
import pytorch_lightning as pl
from filelock import FileLock
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
import os
import torchvision
import clip
from clip_lt.utils.labels_names import labels_names
import torch.nn as nn
from PIL import Image

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load('ViT-B/32', device)
clip_model.eval()

/Users/rotemisraeli/opt/anaconda3/envs/env1/lib/python3.8/site-packages/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [2]:
# dataset_dir_path = '/Volumes/black_ssd/datasets/imagenet_lt/'
dataset_dir_path = '/Users/rotemisraeli/Documents/datasets/TinyImageNet/'

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
])

test_data = torchvision.datasets.ImageFolder(dataset_dir_path+'val/',transform=preprocess)
data_loader = DataLoader(test_data, batch_size=1,num_workers=4)

text_features = torch.load('../text_features.pt')
text_features = text_features / text_features.norm(dim=-1, keepdim=True)
logit_scale = (nn.Parameter(torch.ones([]) * np.log(1 / 0.07))).exp()

In [3]:
good = 0.0
bad = 0.0
with torch.no_grad():
    tqdm_ = tqdm(enumerate(data_loader))
    for i,(image,label) in tqdm_:
        image_features = clip_model.encode_image(image)
        # normalized features
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        # cosine similarity as logits
        logits_per_image = logit_scale * image_features @ text_features.t()
        logits_per_text = logits_per_image.t()
        probs = logits_per_image.softmax(dim=-1)
        predicted = torch.argmax(probs)
        if predicted == label:
            good += 1.0
        bad += 1.0
        tqdm_.desc = f'{100*good/bad}'




0.10117617301125585: 7907it [27:00,  4.88it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/rotemisraeli/opt/anaconda3/envs/env1/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qq/bg7j_rts29s3hwdtklnnfg0c0000gn/T/ipykernel_2564/4205754811.py", line 6, in <module>
    image_features = clip_model.encode_image(image)
  File "/Users/rotemisraeli/opt/anaconda3/envs/env1/lib/python3.8/site-packages/clip/model.py", line 337, in encode_image
    return self.visual(image.type(self.dtype))
  File "/Users/rotemisraeli/opt/anaconda3/envs/env1/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/Users/rotemisraeli/opt/anaconda3/envs/env1/lib/python3.8/site-packages/clip/model.py", line 228, in forward
    x = self.transformer(x)
  File "/Users/rotemisraeli/opt/anaconda3/envs/env1/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1102, in 

TypeError: object of type 'NoneType' has no len()